In [ ]:
    print(f"R Timestamps: {timestamps}")
        
    plt.subplot(1, 2, 1)
    plt.title("Real")
    plt.plot(time, data, label='Signal')
    plt.scatter(timestamps, np.zeros_like(timestamps), marker='o', color='red', label='Real Timestamps', s=10, zorder=5)

    # Plot predicted timestamps
    plt.subplot(1, 2, 2)
    plt.title("Prediction")
    plt.plot(time, data, label='Signal')
    plt.scatter(detected_cough_timestamps, np.zeros_like(detected_cough_timestamps), marker='o', color='red', label='Predicted Timestamps', s=10, zorder=5)

    plt.tight_layout()
    plt.show()
    print()

In [ ]:
# Extraction of high frequency signals (cough/other) from the audio


audio_files = []
audio_files.extend(os.listdir('./audio-records/test-records'))
audio_peak_extractions = dict()

# Removing irrelevant files
for file in audio_files.copy():
    if not file.endswith('.wav'):
        audio_files.remove(file)


for file in audio_files:

    file_path = './audio-records/test-records/' + file

    if file not in audio_peak_extractions:
        audio_peak_extractions[file] = []
    
    # Loading audio_file
    # Target sample rate: 48000
    data, sample_rate = librosa.load(file_path, sr=48000)

    # Audio duration
    duration = librosa.get_duration(path = file_path)
    total_samples = duration * sample_rate
    time = np.arange(0, len(data)) / sample_rate

    # Filtering data
    data = butter_bandpass_filter(data, 1000, 4000, sample_rate, 8)

    # Getting moving average of the data
    moving_avg_data = compute_moving_average(np.abs(data))

    # Normalize the data
    normalized_moving_avg_data = normalize_data(moving_avg_data)
    
    # Getting melspectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=normalized_moving_avg_data, sr=sample_rate)
    mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)    

    frequency_sums = get_frequency_sums(mel_spectrogram_db)
    
    # Detection of high frequency sounds - peaks (Based on frequency domain)
    peak_indices_freq , predicted_timestamps_freq = detect_coughs_freq_domain(frequency_sums, file_path)
    
    # Detection of sounds with high magnitude (Based on time domain)
    peak_indices_time , predicted_timestamps_time = detect_coughs_time_domain(normalized_moving_avg_data)
    
    predicted_timestamps = []
    
    for timestamp in predicted_timestamps_freq:
        
        if any(timestamp - 0.15 < element < timestamp + 0.15 for element in predicted_timestamps_time):
            
            predicted_timestamps.append(timestamp)
        
    
    plt.plot(normalized_moving_avg_data)
    for timestamp in predicted_timestamps:
        
        start = int((timestamp - 0.1) * sample_rate)
        finish = int((timestamp + 0.3) * sample_rate)
        
        plt.scatter([start,finish], [0,0], color='red', zorder=5)    
        audio_peak_extractions[file].append((normalized_moving_avg_data[start:finish], timestamp))
        
    plt.show()

In [1]:

#---------------------------------------------------------------------------------------------------------

# KNN - Only Percentiles
# Define the model

grid_search.fit(X_train_per, y_train_per)
best_params_per = grid_search.best_params_

print("Best Parameters: {}".format(best_params_per))
clf_knn_per = KNeighborsClassifier(**best_params_per)
clf_knn_per.fit(X_train_per, y_train_per)

y_pred_per = clf_knn_per.predict(X_test_per)
cm_per = confusion_matrix(y_test_per, y_pred_per)

f1_per = f1_score(y_test_per, y_pred_per)

print("F1 Score: {}".format(f1_per))

disp = ConfusionMatrixDisplay(confusion_matrix=cm_per, display_labels=display_labels)
disp.plot()
plt.title("KNN - Percentiles")
plt.show()

NameError: name 'grid_search' is not defined

In [ ]:
    
    
    
    """
        copy_data = moving_avg_data.copy()
    coughs = []

    for timestamp in real_timestamps:

        start = int((timestamp - timestamp_range_min) * sample_rate)
        finish = int((timestamp + timestamp_range_max) * sample_rate)
        
        if start < 0:
            start = 0
        
        if finish > int(total_samples):
            finish = int(total_samples) - 1

        data = copy_data[start:finish]
        
        if (data.shape[0] != data_shape):
            continue
        
        coughs.append(data)

    # *******************************
    for cough in coughs:
        # Removing DC component
        cough -= np.mean(cough)
        all_cough_data.append(cough)

        try:
            
            # Find percentile points
            per_25, per_50, per_75, per_90 = find_percentile_points(cough)

            Audiodata['percentile_25'].append(per_25)
            Audiodata['percentile_50'].append(per_50)
            Audiodata['percentile_75'].append(per_75)
            Audiodata['percentile_90'].append(per_90)

            Audiodata_stat['percentile_25'].append(per_25)
            Audiodata_stat['percentile_50'].append(per_50)
            Audiodata_stat['percentile_75'].append(per_75)
            Audiodata_stat['percentile_90'].append(per_90)

            # MFCC Feature extraction
            mfccs, delta_mfccs, delta2_mfccs = get_mfcc_features(cough)

            # Comprehensive MFCCs  
            comprehensive_mfccs = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
            Audiodata['mfcc'].append(comprehensive_mfccs)            
            
            Audiodata_stat['mfcc_max'].append([np.max(mfccs), np.max(delta_mfccs), np.max(delta2_mfccs)])
            Audiodata_stat['mfcc_min'].append([np.min(mfccs), np.min(delta_mfccs), np.min(delta2_mfccs)])
            Audiodata_stat['mfcc_mean'].append([np.mean(mfccs), np.mean(delta_mfccs), np.mean(delta2_mfccs)])
            Audiodata_stat['mfcc_std'].append([np.std(mfccs), np.std(delta_mfccs), np.std(delta2_mfccs)])
            
            # Find spectral centroids
            sc = librosa.feature.spectral_centroid(y=cough, sr=sample_rate)
            Audiodata['sc'].append(sc)
            
            Audiodata_stat['sc_max'].append(np.max(sc))
            Audiodata_stat['sc_min'].append(np.min(sc))
            Audiodata_stat['sc_mean'].append(np.mean(sc))
            Audiodata_stat['sc_std'].append(np.std(sc))
            
            # Add labels
            Audiodata['label'].append('cough')
            Audiodata_stat['label'].append('cough')

        except:
            continue

In [ ]:
  
# ----------------------------------------------------------------------
# Other Data

audio_files_other = os.listdir('./audio-records/no-cough')
timestamp_files_other = os.listdir('./audio-records/no-cough-timestamps')
all_no_cough_data = [] # Will be used to create mel spectrograms

# Filtering irrelevant files
audio_files_other = list(filter(lambda x: x.endswith('.wav'), audio_files_other))
timestamp_files_other = list(filter(lambda x: x.endswith('.txt'), timestamp_files_other))

for file in audio_files_other:

    file_name = file.split(".wav")[0]    
    timestamp_files = [file[0:19] for file in timestamp_files_other]    

    # Finding correspoding timestamp file
    index = timestamp_files.index(file_name)
    timestamp_data = timestamp_files[index]
    
    file_path = './audio-records/no-cough/' + file
    timestamp_path = './audio-records/no-cough-timestamps/' + timestamp_data + '-label.txt'

    # Adding timestamps to the list
    real_timestamps = get_real_timestamps(timestamp_path)    
    
    # Loading audio_file
    # Target sample rate: 48000
    data, sample_rate = librosa.load(file_path, sr=48000)

    # Audio duration
    duration = librosa.get_duration(path = file_path)
    total_samples = duration * sample_rate
    time = np.arange(0, len(data)) / sample_rate

    # Filtering data
    data = butter_bandpass_filter(data, 1000, 4000, sample_rate,8)

    # Getting moving average of the data
    moving_avg_data = compute_moving_average(np.abs(data))

    # Normalize the data
    moving_avg_data = normalize_data(moving_avg_data)
    
    
    # *******************************
    # Find peaks in the data
    copy_data = moving_avg_data.copy()
    peaks = []

    for timestamp in real_timestamps:

        start = int((timestamp - 0.3) * sample_rate)
        finish = int((timestamp + 0.3) * sample_rate)

        if start < 0:
            start = 0
        
        if finish > total_samples:
            finish = total_samples - 1

        data = copy_data[start:finish]
        peaks.append(data)

    # *******************************
    for peak in peaks:
        # Removing DC component
        peak -= np.mean(peak)

        all_no_cough_data.append(peak)

        try:
            
            # Find percentile points
            per_25, per_50, per_75, per_90 = find_percentile_points(peak)

            Audiodata['percentile_25'].append(per_25)
            Audiodata['percentile_50'].append(per_50)
            Audiodata['percentile_75'].append(per_75)
            Audiodata['percentile_90'].append(per_90)

            # MFCC Feature extraction
            mfccs, delta_mfccs, delta2_mfccs = get_mfcc_features(peak)

            print(file_path)
            print(duration)
            print(mfccs.shape)
            print(delta_mfccs.shape)
            print(delta2_mfccs.shape)
            
            # Comprehensive MFCCs
            comprehensive_mfccs = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
            Audiodata['mfcc'].append(comprehensive_mfccs)
            Audiodata['label'].append('other')
            print("Comprehensive mfcc: {}".format(comprehensive_mfccs.shape))
            
            # Find spectral centroids
            sc = librosa.feature.spectral_centroid(y=cough, sr=sample_rate)
            Audiodata['sc'].append(sc)
            print(sc.shape)
            print()

            
        except:
            continue


In [ ]:
# For Audiodata_stat Dictionary

df_stat = pd.DataFrame.from_dict(Audiodata_stat)
print(df_stat.head())

mfcc_stat_max = np.array(df_stat['mfcc_max'].tolist())
mfcc_stat_min = np.array(df_stat['mfcc_min'].tolist())
mfcc_stat_mean = np.array(df_stat['mfcc_mean'].tolist())
mfcc_stat_std = np.array(df_stat['mfcc_std'].tolist())


percentile_values_stat = df_stat.iloc[:, :4].values
sc_values_stat = df_stat.iloc[: , 8:12]

# X and y 
X_stat = np.concatenate((percentile_values_stat, mfcc_stat_max, mfcc_stat_min, mfcc_stat_mean, mfcc_stat_std, sc_values_stat), axis=1)
y_stat = df.iloc[:, -1]


class_mapping = {'cough': 1, 'other': 0}
y_stat = [class_mapping[label] for label in y_stat]

# Training Data
X_train_stat, X_test_stat, y_train_stat, y_test_stat = train_test_split(X_stat, y_stat, random_state= 3, train_size=0.80)

# Scaling
X_train_stat = scale.fit_transform(X_train_stat)
X_test_stat = scale.fit_transform(X_test_stat)

In [ ]:
# Support Vector Machine

# Optimize the parameters
# Finding the best value for gamma and regularization parameter

# C: regularization parameter
param_grid = [
    {
    'C': [0.5, 1, 10, 100],
    'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
    },
]

optimal_params = GridSearchCV(
    SVC(),
    param_grid,
    cv=3,
    scoring='f1'
)

optimal_params.fit(X_train_stat, y_train_stat)
C = optimal_params.best_params_['C']
gamma = optimal_params.best_params_['gamma']
print("Best Parameters: {}".format(optimal_params.best_params_))

# Model Training and Prediction
clf_svm_stat = SVC(random_state=42, C=C, gamma=gamma)
clf_svm_stat.fit(X_train_stat, y_train_stat)

y_pred_stat = clf_svm_stat.predict(X_test_stat)

cm = confusion_matrix(y_test_stat, y_pred_stat)
f1 = f1_score(y_test_stat, y_pred_stat)

print("F1 Score: {}".format(f1))

display_labels = ['Other', 'Cough']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
disp.plot()
plt.title("Support Vector Machine - Percentiles, MFCCs and Spectral Centroids")
plt.show()

In [ ]:
# KNN - Percentiles and MFCCs Combined

# Train the model using the training set
clf_knn_stat = KNeighborsClassifier(p=2)

# Hyperparameter tuning based on the validation performance
param_grid_knn = [
    {
    'n_neighbors': list(range(3, int(len(X_train_stat) * 0.65),2)),
    'weights': ['uniform', 'distance']
    },
]

# GridSearchCV
optimal_params_knn = GridSearchCV(estimator=clf_knn_stat,
                                  param_grid=param_grid_knn,
                                  cv=3,
                                  scoring='f1'
                                )

optimal_params_knn.fit(X_train_stat, y_train_stat)
best_params = optimal_params_knn.best_params_

print("Best Parameters: {}".format(best_params))
clf_knn_stat = KNeighborsClassifier(**best_params)
clf_knn_stat.fit(X_train_stat, y_train_stat)

y_pred_stat = clf_knn_stat.predict(X_test_stat)
cm = confusion_matrix(y_test_stat, y_pred_stat)
f1 = f1_score(y_test_stat, y_pred_stat)

print("F1 Score: {}".format(f1))
display_labels = ['other', 'cough']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
disp.plot()
plt.title("KNN - Percentiles, MFCCs and Spectral Centroids Statistics")
plt.show()

In [ ]:
# File name format
def record_audio(directory):

    current_time = datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%dT%H-%M-%S")

    filename = os.path.join(directory, f"{formatted_time}.wav")

    def timer(duration):
        for remaining in range(duration, 0, -1):
            print(f"Time remaining: {remaining} seconds", end='\r')
            sleep(1)
        print("\nRecording finished.")

    # Sampling frequency
    freq = 48000

    # Recording duration
    duration = 60

    print("Recording started...")
    timer_thread = threading.Thread(target=timer, args=(duration,))
    timer_thread.start()

    recording = sd.rec(int(duration * freq), 
                       samplerate=freq, channels=1, dtype='int16')

    # Record audio for the given number of seconds
    sd.wait()
    timer_thread.join()

    # Convert the NumPy array to audio file
    wv.write(filename, recording, freq, sampwidth=2)
    
    return filename

In [ ]:
# Cough audio loading

# Target sample rate: 48000
filename = record_audio("./audio-records/python-recordings/")
recorded_data, sample_rate = librosa.load(filename, sr=48000)

Audio(filename)

In [ ]:
# Extraction of high frequency signals (cough/other) from the audio
audio_peak_extractions = []

# Filtering data
recorded_data = butter_bandpass_filter(recorded_data, 1000, 4000, sample_rate,8)

# Getting moving average of the data
recorded_data = compute_moving_average(np.abs(recorded_data))

# Scaling data
recorded_data = recorded_data.reshape(-1,1)    
scaler = preprocessing.StandardScaler().fit(recorded_data)
recorded_data = scaler.transform(recorded_data)

# Normalize data
recorded_data = normalize_data(recorded_data)
recorded_data = recorded_data.flatten()
    
# Time 
time = np.arange(0, len(recorded_data)) / sample_rate

# Getting melspectrogram
mel_spectrogram = librosa.feature.melspectrogram(y=recorded_data, sr=sample_rate)
mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)    

frequency_sums = get_frequency_sums(mel_spectrogram_db)

# Detection of high frequency sounds - peaks (Based on frequency domain)
peak_indices_freq , predicted_timestamps_freq = detect_coughs_freq_domain(frequency_sums, filename)
print("Frequency Domain Predicted Timestamps: {}\n".format(predicted_timestamps_freq))

# Detection of sounds with high magnitude (Based on time domain)
peak_indices_time , predicted_timestamps_time = detect_coughs_time_domain(recorded_data)
print("Time Domain Predicted Timestamps: {}".format(predicted_timestamps_time))

predicted_timestamps = []

for timestamp in predicted_timestamps_freq:

    if any(timestamp - 0.15 < element < timestamp + 0.15 for element in predicted_timestamps_time):

        predicted_timestamps.append(timestamp)

print(predicted_timestamps)
plt.plot(time, recorded_data)

for timestamp in predicted_timestamps:

    start = int((timestamp - timestamp_range_min) * sample_rate)
    finish = int((timestamp + timestamp_range_max) * sample_rate)

    if start < 0:
        start = 0

    if finish > len(recorded_data):
        finish = len(recorded_data) - 1
        
    plt.scatter([time[start],time[finish]], [0,0], color='red', zorder=5)    
    extracted_data = recorded_data[start:finish]
    
    if len(extracted_data) < data_shape:

        padding_size = data_shape - len(extracted_data)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        
        extracted_data = np.pad(extracted_data, (left_padding, right_padding), mode='constant', constant_values=0)
    
    elif len(extracted_data) > data_shape:
        extracted_data = extracted_data[:data_shape]
    
    audio_peak_extractions.append((extracted_data, timestamp))

    
plt.show()

In [ ]:
Audiodata_new = {'percentile_25': [], 'percentile_50': [], 'percentile_75': [], 'percentile_90': [], 'mfcc': [], 'sc':[]}

for peak, timestamp in audio_peak_extractions:
    
    # Percentile Calculation
    per_25, per_50, per_75, per_90 = find_percentile_points(peak)


    # MFCC Feature extraction
    mfccs, delta_mfccs, delta2_mfccs = get_mfcc_features(peak)

    # Comprehensive MFCCs
    comprehensive_mfccs = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))

    # Find spectral centroids
    sc = librosa.feature.spectral_centroid(y=peak, sr=sample_rate)
    
    Audiodata_new['percentile_25'].append(per_25)
    Audiodata_new['percentile_50'].append(per_50)
    Audiodata_new['percentile_75'].append(per_75)
    Audiodata_new['percentile_90'].append(per_90)
    Audiodata_new['mfcc'].append(comprehensive_mfccs)
    Audiodata_new['sc'].append(sc)

df_new = pd.DataFrame.from_dict(Audiodata_new)

# Both MFCCs and Percentiles
mfcc_values = np.array(df_new['mfcc'].tolist())
mfcc_values = mfcc_values.reshape(mfcc_values.shape[0], -1)

sc_values = np.array(df_new['sc'].tolist())
sc_values = sc_values.reshape(sc_values.shape[0], -1)

percentile_values = df_new.iloc[:, :4].values

new_data = np.concatenate((percentile_values, mfcc_values, sc_values), axis=1)